Load raster file for tree height.

In [1]:
import gdal
import numpy as np
import nfhelpers as nf

no_data_value = -1

tree_height_file = gdal.Open("SFA_files/TH/TH.vrt")

----------------
Load all the sample areas from Excel-file.

In [2]:
import pandas as pd

sample_areas_file = pd.ExcelFile("SFA_files/provpunkter_svref99.xls")
sheet = sample_areas_file.parse(0)

N_col = sheet["N"]
E_col = sheet["E"]
ID_col = sheet["OBJECTID"]
CLASS_col = sheet["pb"]

sampleAreas = []

for i in range(len(ID_col)):
    sampleAreas.append( nf.SampleArea(ID_col[i], CLASS_col[i], coords=(N_col[i], E_col[i])) )

------------

Extracts data around each sample area coordinate in a given radius.

In [3]:
import nfhelpers as nf
import tqdm
import math
from scipy.ndimage.filters import gaussian_filter
from scipy.ndimage.interpolation import zoom
import matplotlib.pyplot as plt

radius_in_meters = 25

pbar = tqdm.tqdm_notebook(total=len(sampleAreas))

geo_transform = tree_height_file.GetGeoTransform()
radius = radius_in_meters/abs(geo_transform[1])
band = tree_height_file.GetRasterBand(1)
area_buffer = math.ceil(radius) + 5

for sampleArea in sampleAreas:
    point = nf.to_pixel_position(sampleArea.coords, geo_transform)
    cx = point[0]
    cy = point[1]

    area_buffer_CHM = band.ReadAsArray(int(cx)-area_buffer, int(cy)-area_buffer, 2*area_buffer, 2*area_buffer).astype(np.float)

    area_buffer_point = ((cx-int(cx)+area_buffer), (cy-int(cy)+area_buffer))

    circle_mask_low_res = nf.create_circular_mask(area_buffer*2, area_buffer*2, area_buffer_point, radius)
    CHM_values_in_radius = area_buffer_CHM[circle_mask_low_res == True]

    # interpolate data with billinear interpolation to 8x size (2m => 0.25m)
    interpolated_CHM = zoom(area_buffer_CHM, 8, order=1)

    # smooth data with kernel = 1
    smoothed_CHM = gaussian_filter(interpolated_CHM, 1)

    circle_mask_high_res = nf.create_circular_mask(smoothed_CHM.shape[0], smoothed_CHM.shape[1], (area_buffer_point[0]*8, area_buffer_point[1]*8), radius*8)

    cut_out = np.array(smoothed_CHM)
    cut_out[~circle_mask_high_res] = None

    sampleArea.rasters["height"] = CHM_values_in_radius
    sampleArea.rasters["height_smoothed"] = cut_out

    pbar.update(1)

print("Done!")

Done!


-------------

Filters out sample areas with any non-values.

In [4]:
filtered_sampleAreas = []

for sampleArea in sampleAreas:
    if (sampleArea.rasters["height"] == no_data_value).any():
        continue
    filtered_sampleAreas.append(sampleArea)

print(f"Before filtering: {len(sampleAreas)}")
print(f"After filtering: {len(filtered_sampleAreas)}")

Before filtering: 4175
After filtering: 4054


------------

Saves all the extracted data to file.

In [5]:
np.save("numpy_objects/sampleAreas_with_raw_data", filtered_sampleAreas)